In [1]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [2]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入os套件
import os

# 載入time套件
import time

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [4]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 詢問用戶目前身分
    replyJsonPath = "素材/Q目前身分/reply.json"
    result_message_array = []
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
    # 取出消息內用戶的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 呼叫個人用戶名，加入回應裡
    hello = {
        "type": "text",
        "text": "哈囉！很感謝" + str(user_profile.display_name) + "加我為朋友。我知道的東西或許不比你多，但我認識各種職業，在找工作上，可以幫你的忙"
    }
    result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))
        
    # 創建個人資料表 #增加個人ID
    user_ID = user_profile.display_name
    user_data = {
        "user_ID": user_ID,
        "character": [],
        "finding_job": [],
        "good_cv" : [],
        "good_pic" : [],
        "good_con" : [],
        "satisfy_job": [],
        "curious_sal" :[],
        "menu" : []
    }
    
    dn = "user_message/"
    if not os.path.exists(dn):
        os.makedirs(dn)
    fn = str(user_ID) + ".json"
    f = open(dn + fn, "w", encoding="utf-8")
    json.dump(user_data, f)
    f.close()
    
    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )


In [5]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    
    exclude_list = ['在職工作者', '應屆畢業生', '學生', '不告訴你', '很滿意', '還行', '不太滿意', '挺好奇的', '有一點', '不會',
                    '自行探索', '找工作中', '還不急', '沒有', '一定可以', '希望...可以', '不行', '完美', '不敢保證',  '沒什麼信心',
                    '有自信', '還好', '不太有']
    
    if event.message.text in exclude_list:
        pass
    
    # 如果詢問功能建議，針對個人資料進行計分，提供建議
    elif event.message.text == '有建議嗎':
        user_profile = line_bot_api.get_profile(event.source.user_id)
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        with open(dn, "r") as f:
            data = json.load(f)
        
        fun1_score = 0  # 職缺氣象
        fun2_score = 0  # 產業新聞
        fun3_score = 0  # 薪資預測
        fun4_score = 0  # 履歷關鍵字
        fun5_score = 0  # 相由心生
        fun6_score = 0  # 面試捕手
        
        # 目前身分
        if data['character'] == []:
            pass
        elif data['character'][-1] == '在職工作者':
            fun2_score += 1
        elif data['character'][-1] == '應屆畢業生':
            fun1_score += 1
            fun3_score += 1
            fun4_score += 1
            fun5_score += 1
            fun6_score += 1
        elif data['character'][-1] == '學生':
            fun1_score += 1
        
        # 找工作中
        if data['finding_job'] == []:
            pass
        elif data['finding_job'][-1] == '找工作中':
            fun1_score += 1
        elif data['finding_job'][-1] == '還不急':
            fun1_score += 0.5
        else:
            pass
        
        # 履歷信心
        if data['good_cv'] == []:
            pass
        elif data['good_cv'][-1] == '希望...可以':
            fun4_score += 0.5
        elif data['good_cv'][-1] == '不行':
            fun4_score += 1
        else:
            pass
        
        # 照片信心
        if data['good_pic'] == []:
            pass
        elif data['good_pic'][-1] == '不敢保證':
            fun5_score += 0.5
        elif data['good_pic'][-1] == '沒什麼信心':
            fun5_score += 1
        else:
            pass
        
        # 交談信心
        if data['good_con'] == []:
            pass
        elif data['good_con'][-1] == '還好':
            fun6_score += 0.5
        elif data['good_con'][-1] == '不太有':
            fun6_score += 1
        
        # 滿意工作
        if data['satisfy_job'] == []:
            pass
        elif data['satisfy_job'][-1] == '還行':
            fun1_score += 0.5
        elif data['satisfy_job'][-1] == '不太滿意':
            fun1_score += 1
        
        # 好奇薪資
        if data['curious_sal'] == []:
            pass
        elif data['curious_sal'][-1] == '挺好奇的':
            fun3_score += 1
        elif data['curious_sal'][-1] == '有一點':
            fun3_score += 0.5
        
        score_list = [('職缺氣象', fun1_score), ('產業新聞', fun2_score), ('薪資預測', fun3_score), ('履歷關鍵字', fun4_score), ('相由心生', fun5_score), ('面試捕手', fun6_score)]
        score_sorted = sorted(score_list, key=lambda x : x[1], reverse=True)
        
        suggestion = {
        "type": "text",
        "text": "根據您的情況，建議你可先使用以下功能\n1." + score_sorted[0][0] + "\n2." + score_sorted[1][0] + "\n3." + score_sorted[2][0]
        }
        
        result_message_array = TextSendMessage.new_from_json_dict(suggestion)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    else:    
        try:
            # 讀取本地檔案，並轉譯成消息
            result_message_array =[]
            replyJsonPath = "素材/"+event.message.text+"/reply.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

            # 發送
            line_bot_api.reply_message(
                event.reply_token,
                result_message_array
            )
        except:
            sorry = {
                "type": "text",
                "text": "我聽不懂這句話的意思，Sorry!"
            }
            result_message_array = TextSendMessage.new_from_json_dict(sorry)
            line_bot_api.reply_message(
                event.reply_token,
                result_message_array
            )


In [6]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有八個欄位
character, finding_job, good_cv, good_pic, 
good_con, satisfy_job, curious_sal, menu

若folder欄位有值，則
    紀錄個人tag
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    紀錄個人tag
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    
    user_profile = line_bot_api.get_profile(event.source.user_id)

    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'character' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+ query_string_dict.get('character')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["character"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'satisfy_job' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('satisfy_job')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["satisfy_job"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'curious_sal' in query_string_dict:
        
        # 綁定功能選單
        linkRichMenuId = open("素材/"+query_string_dict.get('curious_sal')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        # 詢問是否要功能建議
        replyJsonPath = '素材/'+query_string_dict.get('curious_sal')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["curious_sal"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f) 
              
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    elif 'finding_job' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('finding_job')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["finding_job"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'good_cv' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('good_cv')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["good_cv"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'good_pic' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('good_pic')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["good_pic"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'good_con' in query_string_dict:
    
        # 綁定功能選單
        linkRichMenuId = open("素材/"+query_string_dict.get('good_con')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        # 詢問是否要功能建議
        replyJsonPath = '素材/'+query_string_dict.get('good_con')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["good_con"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f) 
              
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = '素材/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.22.0.2 - - [24/Oct/2019 09:05:52] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:05:54] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:06:03] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:06:04] "POST / HTTP/1.1" 200 -


{'character': ['Q找工作'], 'tag': ['應屆畢業生']}


172.22.0.2 - - [24/Oct/2019 09:06:08] "POST / HTTP/1.1" 200 -


{'character': ['Q找工作'], 'tag': ['學生']}


172.22.0.2 - - [24/Oct/2019 09:06:09] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:06:13] "POST / HTTP/1.1" 200 -


...

172.22.0.2 - - [24/Oct/2019 09:06:38] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:06:41] "POST / HTTP/1.1" 200 -


{'good_con': ['rich_menu_0'], 'tag': ['不太有']}


172.22.0.2 - - [24/Oct/2019 09:06:42] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:06:48] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:06:56] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:19:07] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:20:19] "POST / HTTP/1.1" 200 -


{'menu': ['rich_menu_0_1']}


172.22.0.2 - - [24/Oct/2019 09:20:20] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:20:39] "POST / HTTP/1.1" 200 -
172.22.0.2 - - [24/Oct/2019 09:20:55] "POST / HTTP/1.1" 200 -


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])